In [2]:
from generalization.randomization import available_corruptions

from fastai.vision.all import *

import numpy as np


def get_num_cpus():
    return len(os.sched_getaffinity(0))


def check(learn, dl, plt_title=None):
    y_pred, ys, losses = learn.get_preds(1, dl=dl, with_loss=True, reorder=True)

    print(y_pred[0].min(), y_pred[0].max(), y_pred[0].sum())
    print(y_pred[0].argmax(), ys[0], losses[0])
    plot_losses(losses, width=1, plt_title=plt_title)


def plot_losses(losses, width=1, plt_title=None):
    plt.bar(np.arange(len(losses)), losses, width=width)
    plt.title(plt_title)
    plt.show()


print("Available corruptions:", available_corruptions())

Available corruptions: ['random_labels', 'partial_labels', 'gaussian_pixels', 'random_pixels', 'shuffled_pixels']


## Base Configuration

In [3]:
BATCH_SIZE = 256
NUM_CPUS = get_num_cpus()
LEARNING_RATE = 3e-4
STEPS = 300
PRINT_EVERY = 3
SEED = 5678
EPOCHS = 10

## Configuration for all experiments

In [4]:
from fastai.callback.tensorboard import TensorBoardCallback 
from generalization.randomization import build_cifar10


def build_experiments(corrupt_prob):
    corruptions = available_corruptions()

    experiments = dict()

    for corruption in corruptions:
        if corruption == "gaussian_pixels":
            continue
        dsets, train_loader, test_loader = build_cifar10(
            corruption_name="shuffled_pixels",
            corruption_prob=corrupt_prob,
            batch_size=BATCH_SIZE,
            show_images=False,
            verbose=False,
        )

        _ = dsets[0][100]

        train_loader_ordered = torch.utils.data.DataLoader(
            dsets[0], batch_size=512, shuffle=False, num_workers=get_num_cpus()
        )

        experiments[corruption] = {
            "dsets": dsets,
            "train_loader": train_loader,
            "test_loader": test_loader,
            "train_loader_ordered": train_loader_ordered,
        }
    return experiments


def build_train(train_loader, test_loader, net, log_name):
    tb_cb = TensorBoardCallback(Path("logs") / log_name)    

    data = DataLoaders(train_loader, test_loader)
    learn = Learner(
        data,
        net,
        loss_func=F.cross_entropy,
        opt_func=Adam,
        metrics=[accuracy, top_k_accuracy],
        cbs=[tb_cb],
    )

    return learn


experiments = build_experiments(corrupt_prob=0.3)
experiments["partial_labels"]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


{'dsets': (Dataset CIFAR10
      Number of datapoints: 50000
      Root location: ./data/cifar10
      Split: Train, Corruption: shuffled_pixels,
  Dataset CIFAR10
      Number of datapoints: 10000
      Root location: ./data/cifar10
      Split: Test, Corruption: None),
 'train_loader': <torch.utils.data.dataloader.DataLoader at 0x7f80e2583250>,
 'test_loader': <torch.utils.data.dataloader.DataLoader at 0x7f80e2583400>,
 'train_loader_ordered': <torch.utils.data.dataloader.DataLoader at 0x7f80e2583430>}

In [5]:
from generalization.models import get_cifar_models

models = get_cifar_models(lib="torch")
print(models.keys())

dict_keys(['alexnet', 'inception', 'mlp_1x512', 'mlp_3x512'])


## All Corruptions

In [6]:
import lightning as L
import torchmetrics
from lightning.pytorch.loggers import WandbLogger


class LitModel(L.LightningModule):
    def __init__(self, net: nn.Module, lr: float = 1e-3, n_classes=10):
        super().__init__()
        self.net = net
        self.lr = lr
        self.n_classes = n_classes
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_top5_acc = torchmetrics.Accuracy(
            task="multiclass", num_classes=n_classes, top_k=5
        )

    def forward(self, x):
        out = self.net(x)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("train/loss", loss, on_epoch=True, on_step=False, prog_bar=True)
        self.step_metrics(logits=logits, y=y, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("valid/loss", loss)
        self.step_metrics(logits=logits, y=y, mode="val")
        return loss

    def step_metrics(self, logits, y, mode):
        if mode == "train":
            self.train_acc.update(logits, y)
            self.log(
                "train/acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True
            )

        elif mode == "val":
            self.valid_acc.update(logits, y)
            self.valid_top5_acc.update(logits, y)

            self.log(
                "valid/acc",
                self.valid_acc.compute(),
                on_step=False,
                on_epoch=True,
                prog_bar=True,
            )
            self.log(
                "valid/top5_acc",
                self.valid_top5_acc.compute(),
                on_step=False,
                on_epoch=True,
                prog_bar=True,
            )

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.net.parameters(), lr=0.1, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)
        return [optimizer], [scheduler]

In [8]:
model = "inception"
corruption = "shuffled_pixels"
net = models.get(model)
logger = WandbLogger(
    name=f"{model}",
    project="generalization",
    log_model=True,
    save_dir="logs",
    group=f"{corruption}"
)
trainer = L.Trainer(max_epochs=10, logger=logger, default_root_dir="logs")
pl_model = LitModel(net)
start_time = time.time()
trainer.fit(
    pl_model,
    experiments[corruption]["train_loader"],
    experiments[corruption]["test_loader"],
)

/home/step/.mambaforge/envs/generalization/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | net            | InceptionSmall     | 8.0 M 
1 | train_acc      | MulticlassAccuracy | 0     
2 | valid_acc      | MulticlassAccuracy | 0     
3 | valid_top5_acc | MulticlassAccuracy | 0     
------------------------------------------------------
8.0 M     Trainable params
0         Non-trainable params
8.0 M     Total params
32.191    Total estimated model param

Sanity Checking: 0it [00:00, ?it/s]

Epoch 9: 100%|██████████| 196/196 [00:15<00:00, 12.74it/s, v_num=q18f, valid/acc=0.100, valid/top5_acc=0.500, train/loss=nan.0, train/acc=0.100]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 196/196 [00:15<00:00, 12.64it/s, v_num=q18f, valid/acc=0.100, valid/top5_acc=0.500, train/loss=nan.0, train/acc=0.100]


wandb: Network error (ConnectTimeout), entering retry loop.


In [7]:
import wandb

for model in models.keys():
    print("Model:", model)

    for corruption, exp in experiments.items():
        wandb.finish()
        print("Corruption:", corruption)

        net = models.get(model)
        logger = WandbLogger(
            name=f"{model}",
            project="generalization",
            log_model=True,
            save_dir="logs",
            group=f"{corruption}"
        )
        trainer = L.Trainer(max_epochs=10, logger=logger, default_root_dir="logs")
        pl_model = LitModel(net)
        start_time = time.time()
        trainer.fit(
            pl_model,
            experiments[corruption]["train_loader"],
            experiments[corruption]["test_loader"],
        )
        print(
            f"== Total time: {time.time() - start_time:.3f} s, {(time.time() - start_time)/ 60:.3f} min"
        )
        break

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: alexnet
Corruption: random_labels


wandb: Currently logged in as: stepp1. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | net            | SmallAlexNet       | 56.8 M
1 | train_acc      | MulticlassAccuracy | 0     
2 | valid_acc      | MulticlassAccuracy | 0     
3 | valid_top5_acc | MulticlassAccuracy | 0     
------------------------------------------------------
56.8 M    Trainable params
0         Non-trainable params
56.8 M    Total params
227.307   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 196/196 [00:13<00:00, 14.88it/s, v_num=v27c, valid/acc=0.528, valid/top5_acc=0.931, train/loss=1.130, train/acc=0.602]

...

Corruption: shuffled_pixels


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | net            | MLP                | 1.7 M 
1 | train_acc      | MulticlassAccuracy | 0     
2 | valid_acc      | MulticlassAccuracy | 0     
3 | valid_top5_acc | MulticlassAccuracy | 0     
------------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.935     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 196/196 [00:13<00:00, 14.95it/s, v_num=zv00, valid/acc=0.522, valid/top5_acc=0.922, train/loss=0.525, train/acc=0.815]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 196/196 [00:13<00:00, 14.91it/s, v_num=zv00, valid/acc=0.522, valid/top5_acc=0.922, train/loss=0.525, train/acc=0.815]
== Total time: 133.778 s, 2.230 min
